In [1]:
import numpy as np
import pandas as pd
# import data_collection as dc

import psycopg2
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import LabelEncoder

import assumption_check as ac 

In [2]:
## code to open sql files. Dont need it now

# file_obj = open('../src/sql/05_pull_extr_rpsale_2019.sql')

# file_contents = file_obj.read()
# file_obj.close()

# file_contents
# df_test = pd.read_sql(file_contents, conn)

#### Import data

In [ ]:

dbname = "king_house"

conn = psycopg2.connect(dbname = dbname)

df_sale = pd.read_sql("""
SELECT *

FROM extr_rpsale
""", conn)

df_rdb = pd.read_sql("""
SELECT *

FROM extr_resbldg
""", conn)

df_parcel = pd.read_sql("""
SELECT *

FROM extr_parcel

""", conn)

df_lookup = pd.read_sql("""
SELECT *

FROM look_up

""", conn)


### Create tables

In [ ]:
df_lookup = df_lookup[df_lookup['LUType'] != 'LUType']
df_lookup.head()

In [ ]:
# Selcted variables from the residenstial building dataset:

df_rdb = df_rdb[df_rdb['Major'] != 'Major']
df_rdb['HID'] = df_rdb['Major'].str.zfill(6) + '-'+ df_rdb['Minor'].str.zfill(4)

# columns = ['Major', 'Minor', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor',
#             'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving',
#             'SqFtTotBasement', 'SqFtFinBasement', 'FinBasementGrade','SqFtGarageBasement', 
#             'SqFtGarageAttached', 'DaylightBasement','SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck',
#             'Bedrooms','BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'HID']
# df_rdb_set = df_rdb.loc[:, columns]
# df_rdb_set.head()

In [ ]:
df_rdb.columns

In [ ]:
# Selected variables from the parcel dataset:
df_parcel = df_parcel[df_parcel['Major'] != 'Major']
df_parcel['HID'] = df_parcel['Major'].str.zfill(6) + '-' + df_parcel['Minor'].str.zfill(4)

# columns = ['Major', 'Minor', 'SqFtLot','WfntLocation', 
#            'TrafficNoise', 'AirportNoise', 'PowerLines', 'OtherNuisances', 'HID'] 
# df_parcel_set = df_parcel.loc[:, columns]
df_parcel.shape

#'PugetSound', 'LakeWashington','LakeSammamish', 'SmallLakeRiverCreek',

In [ ]:
# Selseted variables from the property sale dataset: 

df_sale = df_sale[df_sale['Major'] != 'Major']

df_sale['HID'] = df_sale['Major'].str.zfill(6) + '-' + df_sale['Minor'].str.zfill(4)
df_sale_2019_set = df_sale[df_sale['DocumentDate'].str.contains('2019')]

# columns = ['Major', 'Minor', 'DocumentDate', 'SalePrice', 'PropertyType', 'HID']
# df_sale_2019_set = df_sale_2019.loc[:, columns]
# df_sale_2019_.head()
df_sale_2019_set.shape

In [ ]:
# Join 3 datasets together

df_combine = df_sale_2019_set.merge(df_parcel, how='inner', on='HID')
df_combine = df_combine.merge(df_rdb, how='inner', on='HID')

In [ ]:
df_combine.columns
df_combine.shape

In [ ]:
king_house_2019 = df_combine

In [ ]:
king_house_2019 = king_house_2019.astype(
                     {'SalePrice':'float',
                      'SqFtLot':'float',  
                      'SqFt1stFloor':'float', 
                       'SqFtHalfFloor':'float',
                       'SqFt2ndFloor':'float', 
                       'SqFtUpperFloor':'float', 
                       'SqFtUnfinFull':'float', 
                       'SqFtUnfinHalf':'float',
                       'SqFtTotLiving':'float', 
                       'SqFtTotBasement':'float', 
                       'SqFtFinBasement':'float',
                       'FinBasementGrade':'float', 
                       'SqFtGarageBasement':'float', 
                       'SqFtGarageAttached':'float',
                       'SqFtOpenPorch':'float', 
                       'SqFtEnclosedPorch':'float', 
                       'SqFtDeck':'float',
                       'Bedrooms':'float', 
                       'BathHalfCount':'float', 
                       'Bath3qtrCount':'float', 
                       'BathFullCount':'float',
                       'TrafficNoise': 'int', 
                       'AirportNoise':'float',
                       }
                      )

In [ ]:
# king_house_2019.head()

In [ ]:
cols = list(king_house_2019.columns)
cols = [cols[2]] + cols[:2] + cols[3:]
king_house_2019 = king_house_2019[cols]
king_house_2019 = king_house_2019[(
                                   (king_house_2019['PropertyType'] == '11') 
                                  |(king_house_2019['PropertyType'] == '12') 
                                  |(king_house_2019['PropertyType'] == '13') 
                                  |(king_house_2019['PropertyType'] == '14') 
                                )
                                  & (king_house_2019.SalePrice >= 50000)]

In [ ]:
# king_house_2019 = king_house_2019.drop(columns = ['Major_x', 'Major_y', 'Minor_x', 'Minor_y', 'DocumentDate', 'Major', 'Minor'])
# king_house_2019 = king_house_2019.drop(columns = 'DocumentData')

In [ ]:
king_house_2019.iloc[:, 13:30].head(20)

king_house_2019['SqlTotal'] = king_house_2019['SqFt1stFloor'] + king_house_2019['SqFtHalfFloor'] + king_house_2019['SqFt2ndFloor']\
                         + king_house_2019['SqFtUpperFloor'] + king_house_2019['SqFtUnfinFull'] + king_house_2019['SqFtUnfinHalf']\
                         + king_house_2019['SqFtTotBasement'] + king_house_2019['SqFtFinBasement'] + king_house_2019['FinBasementGrade']\
                         + king_house_2019['SqFtGarageBasement'] + king_house_2019['SqFtGarageAttached'] \
                         + king_house_2019['SqFtOpenPorch'] + king_house_2019['SqFtEnclosedPorch'] + king_house_2019['SqFtDeck']

king_house_2019['pwrlines'] = king_house_2019['PowerLines'] == 'Y'
king_house_2019['othernuisance'] = king_house_2019['OtherNuisances']=='Y'

In [ ]:
king_house_2019['nuisance_total'] = (
    king_house_2019['AirportNoise'] +
    king_house_2019['TrafficNoise'] +
    king_house_2019['pwrlines'] +
    king_house_2019['othernuisance']
)

In [ ]:
king_house_2019['PorchTotal']=king_house_2019.SqFtOpenPorch + king_house_2019.SqFtEnclosedPorch

In [ ]:
king_house_2019['is_waterfront'] = (king_house_2019['WfntLocation'] != '0').astype(int)

In [ ]:
king_house_2019['BathTotal'] = king_house_2019['BathHalfCount']*0.5 + king_house_2019['Bath3qtrCount'] * 0.75 + king_house_2019['BathFullCount']

In [ ]:
# df_lookup_w = df_lookup[df_lookup['LUType']=='1  ']

# df_lookup_w.head(10)

In [ ]:
king_house_2019['PorchTotal'] = king_house_2019['SqFtOpenPorch'] + king_house_2019['SqFtEnclosedPorch']
king_house_2019['is_porch'] = (king_house_2019['PorchTotal']==0).astype('int')

In [ ]:
king_house_2019.columns

In [ ]:
columns = ['SalePrice','SqFtTotLiving', 'Bedrooms', 'SqlTotal', 'BathTotal']
df = king_house_2019[columns]
colrelation = ac.correlation_fig(df)

In [ ]:
columns = ['SalePrice', 'SqlTotal']
df = king_house_2019

In [ ]:
kh_base = ac.create_df(df, columns)

kh_base_model = ac.create_model(kh_base)

model_summary = ac.model_summary(kh_base_model)
model_summary

### Check liniear regression assumtions linearity:

In [ ]:
check = ac.linearity_check(kh_base_model)

### Check Normality

The Jarque-Bera test is performed automatically as part of the model summary output, labeled Jarque-Bera (JB) and Prob(JB).

The null hypothesis is that the residuals are normally distributed, alternative hypothesis is that they are not. Thus returning a low p-value means that the current model violates the normality assumption.

In [ ]:
ac.normality_check()

### Check Homoscadasticity:
Linear regression assumes that the variance of the dependent variable is homogeneous across different value of the independent variable(s). We can visualize this by looking at the predicted dependent variable values vs. the residuals.

In [ ]:
ac.homosdt_check_fig(kh_base, kh_base_model)


In [ ]:
ac.homosdt_check_test(kh_base, kh_base_model)

### Check Independence:
The independence assumption means that the independent variables must not be too collinear. If we have only one independent variable, so we don't need to check this.

In [ ]:
# model 2: add new second variable is_water:
columns = ['SalePrice', 'SqlTotal', 'BathTotal']
df = king_house_2019

kh_model2 = ac.create_df(df, columns)

kh_model2.head()

In [ ]:
model2 = ac.create_model(kh_model2)

In [ ]:
ac.model_summary(model2)

In [ ]:
ac.linearity_check(model2)

In [ ]:
ac.normality_check()

In [ ]:
ac.homosdt_check_fig(kh_model2, model2)

In [ ]:
ac.homosdt_check_test(kh_model2, model2)

In [ ]:


def independence_check(df):

    rows = df.iloc[:, 1:].values

    vif_df = pd.DataFrame()
    vif_df["VIF"] = [variance_inflation_factor(rows, i) for i in range(len(df.columns)-1)]
    vif_df["feature"] = list(df.columns[1:])

    return vif_df

independence_check(kh_model2)

In [ ]:
ac.independence_check(kh_model2)